In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from  ift6758.data.data_acquisition import Season
season2018 = Season(2018,"../ift6758/data")
df_season2018_clean = season2018.clean_data()
# df[["gamePk","liveData.plays.allPlays"]]



File already Exists, loading from ../ift6758/data/PICKLE//2018_clean.pkl


In [5]:
df_season2018_clean.head()


,result.event,gamePk,team.name,about.period,about.periodTime,about.periodType,about.periodTimeRemaining,coordinates.x,coordinates.y,result.secondaryType,result.emptyNet,result.strength.name,shooter,goalie
0,Shot,2018010002,Washington Capitals,1,00:25,REGULAR,19:35,-48.0,6.0,Snap Shot,NaN,NaN,Andre Burakovsky,Zane McIntyre
1,Shot,2018010002,Boston Bruins,1,03:37,REGULAR,16:23,61.0,9.0,Wrist Shot,NaN,NaN,Danton Heinen,Pheonix Copley
2,Shot,2018010002,Boston Bruins,1,04:05,REGULAR,15:55,72.0,5.0,Backhand,NaN,NaN,David Krejci,Pheonix Copley
3,Shot,2018010002,Boston Bruins,1,04:23,REGULAR,15:37,33.0,-12.0,Wrist Shot,NaN,NaN,Mark Fayne,Pheonix Copley
4,Shot,2018010002,Boston Bruins,1,04:32,REGULAR,15:28,35.0,-6.0,Snap Shot,NaN,NaN,Anton Blidh,Pheonix Copley


In [ ]:
df_season2018_clean